In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report


In [44]:
train = np.load('fashion_train.npy')
test = np.load('fashion_test.npy')

In [45]:
def find_x_y(data):
    Y = data[:, -1]
    X = np.delete(data, -1, axis=1)
    return X, Y

label_dict = {0: 'T_shirts', 1: 'Trouseres', 2: 'Pullover', 3: 'Dress', 4: 'Shirt'}

train_x, train_y = find_x_y(train)
test_x, test_y = find_x_y(test)

def compute_class_weights(y):
    class_counts = np.bincount(y)
    total_samples = len(y)
    class_weights = total_samples / (len(class_counts) * class_counts)
    return class_weights

# Flatten the class labels to avoid the TypeError
flat_train_y = train_y.flatten()
class_weights = compute_class_weights(flat_train_y)
class_weights_dict = dict(enumerate(class_weights))

train_x = train_x.reshape((10000, 28, 28, 1))
train_x = train_x.astype('float32') / 255

test_x = test_x.reshape((5000, 28, 28, 1))
test_x = test_x.astype('float32') / 255

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)



In [46]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dense(5, activation='softmax'))
#model.add(layers.BatchNormalization())
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 13, 13, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 3, 3, 128)         73856     
                                                                 
 flatten_5 (Flatten)         (None, 1152)             

In [47]:
# Compiling the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
def lr_schedule(epoch):
    """
    Returns a custom learning rate that decreases as epochs progress.
    """
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * drop ** (epoch / epochs_drop)
    return lr

# Set up the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [49]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#Set up a ModelCheckpoint to save the best model during training
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Train the model with early stopping
history = model.fit(train_x, train_y, epochs=50, batch_size=32, validation_data=(test_x, test_y), callbacks=[lr_scheduler,early_stopping,checkpoint])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_x, test_y)
print('Test accuracy:', test_acc)

Epoch 1/50
312/313 [============================>.] - ETA: 0s - loss: 0.6179 - accuracy: 0.7465
Epoch 1: val_accuracy improved from -inf to 0.82840, saving model to best_model.h5
313/313 [==============================] - 5s 13ms/step - loss: 0.6175 - accuracy: 0.7467 - val_loss: 0.4531 - val_accuracy: 0.8284 - lr: 0.0010
Epoch 2/50
 11/313 [>.............................] - ETA: 3s - loss: 0.4178 - accuracy: 0.8381

C:\Users\kubic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


312/313 [============================>.] - ETA: 0s - loss: 0.3961 - accuracy: 0.8509
Epoch 2: val_accuracy improved from 0.82840 to 0.84900, saving model to best_model.h5
313/313 [==============================] - 4s 11ms/step - loss: 0.3959 - accuracy: 0.8510 - val_loss: 0.4041 - val_accuracy: 0.8490 - lr: 8.7055e-04
Epoch 3/50
308/313 [============================>.] - ETA: 0s - loss: 0.3313 - accuracy: 0.8741
Epoch 3: val_accuracy improved from 0.84900 to 0.86060, saving model to best_model.h5
313/313 [==============================] - 4s 11ms/step - loss: 0.3319 - accuracy: 0.8736 - val_loss: 0.3823 - val_accuracy: 0.8606 - lr: 7.5786e-04
Epoch 4/50
308/313 [============================>.] - ETA: 0s - loss: 0.2958 - accuracy: 0.8846
Epoch 4: val_accuracy did not improve from 0.86060
313/313 [==============================] - 4s 12ms/step - loss: 0.2968 - accuracy: 0.8836 - val_loss: 0.3997 - val_accuracy: 0.8460 - lr: 6.5975e-04
Epoch 5/50
311/313 [============================>.] -

In [50]:


# Predict on the test set
y_pred = model.predict(test_x)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded true labels to class indices
y_true_classes = np.argmax(test_y, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes))

157/157 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.88      0.79      0.83      1000
           1       0.99      0.97      0.98      1000
           2       0.89      0.87      0.88      1000
           3       0.93      0.92      0.92      1000
           4       0.70      0.81      0.75      1000

    accuracy                           0.87      5000
   macro avg       0.88      0.87      0.87      5000
weighted avg       0.88      0.87      0.87      5000

